In [32]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score
from pprint import pprint
from string import punctuation
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [33]:
class EP:
    
    def __init__(self, ethnicity):  
        
        assert ethnicity in 'iranian polish italian german japanese vietnamese'.split(), f'incorrect ethnicity: {ethnicity}!'
        
        datafile = f'/Users/ik/Data/names/training-{ethnicity}.csv'
        self.target_col = f'is_{ethnicity}'
        
        self.data = pd.read_csv(datafile)
        
        assert self.target_col in self.data.columns, f'there is no {self.target_col} in data file!'  
        
        try:
            self.NODOUBT_FIRST_NAMES = list({line.strip() for line in open(f'/Users/ik/Data/names/real_{ethnicity}.txt','r').readlines() if line.strip()})
        except:
            self.NODOUBT_FIRST_NAMES = []
        
    def add_nodoubt_names(self):
        
        print(f'adding {len(self.NODOUBT_FIRST_NAMES)} first names..')
        
        self.data = pd.concat([self.data, pd.DataFrame({'full_name': self.NODOUBT_FIRST_NAMES, 
                                                        self.target_col: [1]*len(self.NODOUBT_FIRST_NAMES)})], 
                              ignore_index=True).sample(frac=1.)
        return self

In [34]:
class Selector(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return '_start_' + x[self.col_name] + '_end_'

class WordCount(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: len(_.split())).values.reshape(x.shape[0],1)
        return res

class NameLength(BaseEstimator, TransformerMixin):
    """
    return the length of the full name
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.str.len().values.reshape(x.shape[0],1)
        return res

class FirstLast(BaseEstimator, TransformerMixin):
    """
    is the first word longer than the last one
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: np.argmax([len(p) for i, p in enumerate(_.split()) if i in [0,len(_.split())-1]])).values.reshape(x.shape[0],1)
        return res

In [35]:
if __name__ == '__main__':
    
    ic = EP('vietnamese')
    #.add_nodoubt_names()
    
    # split into the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ic.data.drop([ic.target_col], axis=1), 
                                                        ic.data[ic.target_col],  test_size=0.2, 
                                                        random_state=391, stratify=ic.data[ic.target_col])
    

    pipe = Pipeline([('select_fullname', Selector('full_name')),  # df with 1 column
                        ('features', FeatureUnion(
                             [('char_level', CountVectorizer(strip_accents='ascii', analyzer='char', 
                                                          ngram_range=(1, 4))),
                              ('word_level', CountVectorizer(strip_accents='ascii', analyzer='word',
                                                       ngram_range=(1,2))),
                              ('word_count', WordCount()),
                              ('full_name_length', NameLength()),
                              ('firstlast', FirstLast())],
                                 transformer_weights={
                                            'char_level': 0.8,
                                            'word_level': 0.5,
                                            'word_lengths': 1.0,
                                                        })
                     ),
                     ('normalise', Normalizer()),
                     ('pca', TruncatedSVD(n_components=120)),
                    ('clf', SGDClassifier(max_iter=1000))])
    
    param_grid = {'features__transformer_weights': [{'char_level': 0.2,
                                                     'word_level': 0.1,
                                                     'word_count':0.4,
                                                    'full_name_length': 0.5}, 
                                                   {'char_level': 0.9,
                                                     'word_level': 0.2,
                                                     'word_lengths':0.9,
                                                   'full_name_length': 0.3},
                                                   {'char_level': 0.9,
                                                     'word_level': 0.3,
                                                     'word_count':0.2,
                                                   'full_name_length': 0.7}],
                    "clf__loss": ['hinge', 'modified_huber', 'log', 'perceptron'],
                     'clf__penalty': ['l1','l2','elasticnet']}
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid, verbose=10)
    
    grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9757462686567164, total=   1.1s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9681647940074907, total=   1.0s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.2s remaining:    0.0s


[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9550561797752809, total=   1.0s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 


...

[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.9569288389513109, total=   1.0s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.9776119402985075, total=   1.0s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.9775280898876404, total=   1.0s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transfo

[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9606741573033708, total=   1.6s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9550561797752809, total=   1.6s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.9794776119402985, total=   1.5s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_length

[CV]  clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.9756554307116105, total=   1.2s
[CV] clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.9625468164794008, total=   1.2s
[CV] clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.9626865671641791, total=   0.9s
[CV] clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_lev

[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.9776119402985075, total=   1.0s
[CV] clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.9644194756554307, total=   0.9s
[CV] clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.9588014981273408, total=   0.9s
[CV] clf__loss=perceptron, clf__penal

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed:  2.0min finished


In [36]:
print(classification_report(y_test, grid_search.predict(X_test)))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98       236
          1       1.00      0.95      0.98       165

avg / total       0.98      0.98      0.98       401



In [44]:
test_data = pd.DataFrame({'full_name': ['han solo', 'tomoko nguyen', 'chan','kazuo ishuguro', 'mohammad eli', 'mehrdad','myriam',
                                         'nejad lowson','hosein', 'xi','babak', 'igor', 'ai tran', 'david perron',
                                        'andrzej woods', 'fabian miller', 'kumar', 'otto floms', 'suzuki', 'toyota', 
                                        'honda', 'kawasaki', 'james tran', 'bo nguyen', 'subaru', 'ahmad lopes zeto', 'bob', 'mario razzi', 'john reed']})

test_data['prediction'] = grid_search.predict(test_data)
test_data['prediction'] = test_data['prediction'].apply(lambda _: 'yes' if _ else 'no')

In [45]:
test_data

,full_name,prediction
0,han solo,no
1,tomoko nguyen,yes
2,chan,no
3,kazuo ishuguro,no
4,mohammad eli,no
5,mehrdad,no
6,myriam,no
7,nejad lowson,no
8,hosein,no
9,xi,no


In [39]:
grid_search.best_params_

{'clf__loss': 'modified_huber',
 'clf__penalty': 'l2',
 'features__transformer_weights': {'char_level': 0.9,
  'full_name_length': 0.7,
  'word_count': 0.2,
  'word_level': 0.3}}